In [ ]:
# This notebook tries to calculate a quantity presented in the video,
# https://www.youtube.com/watch?v=hlM7zdf7zwU&feature=share , 
# "Confidence intervals and margin of error | AP Statistics | Khan Academy"
#
# Given a population of size M, 
# and if the population is split into Group A of size Q,
# and Group B makes up , M - Q,
# , such that p = Q/M
# and if we try to sample with a sample size of n,
#
# , what is the probability, using a sample distribution around p, 
# with sample_sigma = sqrt((p)*(1 - p)/n)
# , that for the sample, the sample proportion observed of group  A, will be, 
# within 2 sample_sigma, of p ?
#
# The general approach here is , initially, to try to use combinatorial math,
# to count how many ways are there of selecting different proportions, in a sample,
# from Group A and Group B.


In [37]:
from scipy.special import comb
import math
import numpy as np
from decimal import Decimal

In [35]:
import ipdb

In [5]:
from __future__ import division

In [44]:
parameters = {'M': 100000, # too big !
             'n': 100,
             'Q': 46402}

parameters = {'M': 10000,
             'n': 100,
             'Q': 4640}

In [58]:
def get_sample_sigma(p, n):
    return math.sqrt((p*(1-p)/n))

In [59]:
get_sample_sigma(9/20, 10)

0.15732132722552272

In [45]:
get_sample_sigma(4640/10000, 100)

0.04987023160162784

In [51]:
## For an arbitrary range, (p - 2*sigma_sample_p , p + 2*sigma_sample_p)
# 
sigma_sample_p = get_sample_sigma(4640/10000, 100)
Q = parameters['Q']; M = parameters['M']; p = Q/M
n = parameters['n']
print p, p*n
print Q - 2*sigma_sample_p*n, Q + 2*sigma_sample_p*n
print p*n - 2*sigma_sample_p*n, p*n + 2*sigma_sample_p*n

0.464 46.4
4630.02595368 4649.97404632
36.4259536797 56.3740463203


In [64]:
def make_2sigma_Q_range(params):
    M = params['M']
    n = params['n']
    Q = params['Q']
    p = Q/M ; assert p != 0

    sigma_sample_p = get_sample_sigma(p, n)
    range1 = [p - 2*sigma_sample_p, p + 2*sigma_sample_p]
    range2 = [int(np.floor(x*n)) for x in range1]
    return {'range1': range1, 'range2': range2}
    

In [53]:
make_2sigma_Q_range(parameters)

[36, 56]

In [56]:
def calculate_probability_that_sample_prob_is_in_range(params, some_range, ):
    M = params['M']
    n = params['n']
    Q = params['Q']
    p = Q/M ; assert p != 0
    sigma = get_sample_sigma(p, n)
    a, b = some_range
    
    
    denominator = comb(M, n)
    
    numerator = np.sum([
        comb(M - i, n - i) * comb(Q, i)
        for i in range(a, b)
    ])
    
    return numerator/denominator

In [57]:
some_range = make_2sigma_Q_range(parameters)

a, b = some_range
print a,b
# prob = ipdb.runcall(calculate_probability_that_sample_prob_is_in_range,               
#                    parameters, some_range)
prob = calculate_probability_that_sample_prob_is_in_range(
                    parameters, some_range)
print prob

36 56
6775154660566004.0


In [65]:
# Hmm. Solve on super small scale first , 
print "comb(20,10) = ", comb(20,10)
smallparams = {'M': 20, 'Q':9, 'n':10}
print make_2sigma_Q_range(smallparams)

comb(20,10) =  184756.0
{'range2': [1, 7], 'range1': [0.13535734554895457, 0.7646426544510454]}


In [66]:
# is my numerator right in counting the sampling ways?
np.sum([comb(20-i,10-i)*comb(9,i) for i in range(10)])

5707449.0

In [71]:
# Dang that doesn't look right at all, 5707449.0 >> 184756.0 
# hmm, 
[[i,(20-i,10-i),(9,i)] for i in range(10)]

[[0, (20, 10), (9, 0)],
 [1, (19, 9), (9, 1)],
 [2, (18, 8), (9, 2)],
 [3, (17, 7), (9, 3)],
 [4, (16, 6), (9, 4)],
 [5, (15, 5), (9, 5)],
 [6, (14, 4), (9, 6)],
 [7, (13, 3), (9, 7)],
 [8, (12, 2), (9, 8)],
 [9, (11, 1), (9, 9)]]

In [113]:
# Hm okay, my denominator was probably wrong all along... 🤔
# ok... try a different version of this function ..

def calculate_probability_that_sample_prob_is_in_range(params, some_range, ):
    M = params['M']
    n = params['n']
    Q = params['Q']
    p = Q/M ; assert p != 0
    sigma = get_sample_sigma(p, n)
    a, b = some_range
    
    assert 0 < a < b < n
    
    denominator = np.sum([
        comb(M - i, n - i) * comb(Q, i)
        for i in range(0, n)
    ])
    
    numerator = np.sum([
        comb(M - i, n - i) * comb(Q, i)
        for i in range(a, b)
    ])
    print 'numerator/denominator = %s/%s' % (numerator, denominator)
    
    return numerator/denominator


# def count_ways_to_choose_group_a_and_b

In [99]:
print 'small params, ', smallparams
sample_range_dict = make_2sigma_Q_range(smallparams)
sample_prob_range, variable_range = sample_range_dict['range1'], sample_range_dict['range2']
print 'sample_prob_range, ', sample_prob_range
print 'variable_range, ', variable_range

prob = calculate_probability_that_sample_prob_is_in_range(
                    smallparams, variable_range)
print 'probability that when sampling n from M, that the proportion of the sample'
print 'probability, is within two sigma of the actual group probability'
print ', prob = ', prob

small params,  {'Q': 9, 'M': 20, 'n': 10}
sample_prob_range,  [0.13535734554895457, 0.7646426544510454]
variable_range,  [1, 7]
numerator/denominator = 5511792.0/5707449.0
probability that when sampling n from M, that the proportion of the sample
probability, is within two sigma of the actual group probability
, prob =  0.9657190103669783


In [114]:
# 
# Nice, finally getting a good looking number. ^^.
# Maybe this result is 0.965 and not 0.95 , because my parameters are in the toyrange.

# next, try with the large numbers again ...
parameters100k = {'M': 100000, # too big !
             'n': 100,
             'Q': 46402}
parameters10k = {'M': 10000,
             'n': 1000,
             'Q': 4640}

sample_range_dict = make_2sigma_Q_range(parameters10k)
sample_prob_range, variable_range = sample_range_dict['range1'], sample_range_dict['range2']
range_size = sample_prob_range[1] - sample_prob_range[0]
print 'sample_prob_range, ', sample_prob_range, ', size: ', range_size
print 'variable_range, ', variable_range

prob = calculate_probability_that_sample_prob_is_in_range(
                    parameters10k, variable_range)
print ', prob = ', prob



sample_prob_range,  [0.43245929613974987, 0.4955407038602502] , size:  0.0630814077205
variable_range,  [432, 495]
numerator/denominator = inf/inf
, prob =  nan


/usr/local/miniconda3/envs/blogplt2/lib/python2.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in double_scalars


In [88]:
# Grr, dang still getting a `nan` as an output, like the numbers are just too big?

In [117]:
comb(long(10000), long(1000))

inf

In [120]:
# Okay, try again with this module called gmpy..
#
#
#

In [118]:
import gmpy

In [119]:
def calculate_probability_that_sample_prob_is_in_range__gmpy(params, some_range, ):
    M = params['M']
    n = params['n']
    Q = params['Q']
    p = Q/M ; assert p != 0
    sigma = get_sample_sigma(p, n)
    a, b = some_range
    
    assert 0 < a < b < n
    
    denominator = np.sum([
        gmpy.comb(M - i, n - i) * gmpy.comb(Q, i)
        for i in range(0, n)
    ])
    
    numerator = np.sum([
        gmpy.comb(M - i, n - i) * gmpy.comb(Q, i)
        for i in range(a, b)
    ])
    print 'numerator/denominator = %s/%s' % (numerator, denominator)
    
    return numerator/denominator



In [122]:
parameters100k = {'M': 100000, # too big !
             'n': 100,
             'Q': 46402}
parameters10k = {'M': 10000,
             'n': 5000,
             'Q': 4640}

sample_range_dict = make_2sigma_Q_range(parameters10k)
sample_prob_range, variable_range = sample_range_dict['range1'], sample_range_dict['range2']
range_size = sample_prob_range[1] - sample_prob_range[0]
print 'sample_prob_range, ', sample_prob_range, ', size: ', range_size
print 'variable_range, ', variable_range

prob = calculate_probability_that_sample_prob_is_in_range__gmpy(
                    parameters10k, variable_range)
print ', prob = ', prob



sample_prob_range,  [0.44989456842205816, 0.4781054315779419] , size:  0.0282108631559
variable_range,  [2249, 2390]
numerator/denominator = 4066462602278825629295955468908316588331308573334800880125419211591883356352132284690645407329794557316940930028727005016729002016391168805912202071998365917186401893487116182843410722912519050510411678232318053590094818063194414299437535844724578267163225076994888935233161521589968956878097056869240175633585744623911249378464761794478979032413520562049899373478167418214524441644841646843389786833223275028783452498616295173804209666908025267382315433747178117789114350248438615072222789389377559216015658847365284069018143462465668969123565043808195526408028019590699866544420245101381832829975345560426088102587982925712472572740184133703302195198915558115836630896368663491342433369932242119519379982435949770860033905133553356995246863261801810923718044441598047205764812747529812848957360112711570784967082907611675401623257750980648022378647459249550

In [ ]:
# Hmm ok, so this module clearly handles the numbers nicely, 
# but there must be some flaw in my calculation..